In [1]:
import pandas as pd
import datetime
from lxml import etree
import glob
xml_elements = ['datecreated', 'title', 'articletitle', 'abstracttext', 'language',  'country', 'publicationstatus']

In [2]:
abstractlist = glob.glob("abstracts/*")

In [22]:
def parse_date(time):
    time = time.getchildren()
    cal = dict((v,k) for k,v in enumerate(calendar.month_abbr))
    try:
        date = datetime.date(int(time[0].text), int(time[1].text), int(time[2].text))
    except Exception:
        #print cal(time[1].text)
        date = datetime.date(int(time[0].text), int(cal[time[1].text.strip()]), int(time[2].text))
    return date

def create_df(articlelist):
    findelems = lambda arr, elem: [item[1] for item in arr if item[0] == elem]
    stripelems =  lambda arr: [item.text.strip() for item in arr]
    
    df = pd.DataFrame(columns=xml_elements)
    
    for article in range(len(articlelist)):
        print article
        print articlelist[article]
        article_dict = dict()

        context = list(etree.iterparse(articlelist[article], events=("end",), tag=xml_elements))
        context = [(_[1].tag, _[1]) for _ in context]

        article_dict['abstracttext'] = ' '.join(stripelems(findelems(context, 'abstracttext')))
        try:
            article_dict['datecreated'] = parse_date(findelems(context, 'datecreated')[0])
        except Exception:
            article_dict['datecreated'] = 'not available'
        article_dict['title'] = ' '.join(stripelems(findelems(context, 'title')))
        article_dict['articletitle'] = ' '.join(stripelems(findelems(context, 'articletitle')))
        article_dict['language'] = ' '.join(stripelems(findelems(context, 'language')))
        article_dict['publicationstatus'] = ' '.join(stripelems(findelems(context, 'publicationstatus')))

        article_dict['country'] = ' '.join(set([_.lower() for _ in stripelems(findelems(context, 'country'))]))
        
        for key in article_dict.keys():
            df.loc[article,key] = article_dict[key]
    return df

In [23]:
qq = create_df(abstractlist)

0
abstracts/11092281.xml
1
abstracts/8611058.xml
2
abstracts/14578243.xml
3
abstracts/11336416.xml
4
abstracts/17091409.xml
5
abstracts/7674514.xml
6
abstracts/14973999.xml
7
abstracts/11330653.xml
8
abstracts/10906651.xml
9
abstracts/10465168.xml
10
abstracts/12585994.xml
11
abstracts/15070788.xml
12
abstracts/8445916.xml
13
abstracts/12537734.xml
14
abstracts/12662389.xml
15
abstracts/12432042.xml
16
abstracts/9164801.xml
17
abstracts/9861593.xml
18
abstracts/8621845.xml
19
abstracts/15865677.xml
20
abstracts/11327435.xml
21
abstracts/12502618.xml
22
abstracts/17071428.xml
23
abstracts/15007010.xml
24
abstracts/9809732.xml
25
abstracts/12118972.xml
26
abstracts/7103152.xml
27
abstracts/11130213.xml
28
abstracts/12202466.xml
29
abstracts/8877239.xml
30
abstracts/8902660.xml
31
abstracts/8421475.xml
32
abstracts/3311051.xml
33
abstracts/9892589.xml
34
abstracts/8572664.xml
35
abstracts/8790035.xml
36
abstracts/12603963.xml
37
abstracts/2656959.xml
38
abstracts/15266527.xml
39
abstracts

In [26]:
qq[qq['datecreated'] == 'not available'] 

,datecreated,title,articletitle,abstracttext,language,country,publicationstatus
907,not available,,,PURPOSE: Both the US Preventive Services Task ...,eng,,ppublish
2071,not available,,,This report represents a departure from the ch...,eng,,ppublish
2518,not available,,,The aim of this guideline is to offer best pra...,eng,,ppublish
